# Estudo da composição das previsões do rating final

O objetivo é realizar previsões para toda base gerando um rating unindo o contribuinte e posteriormente o igr gerado na regressão

In [9]:
import os
import dotenv
import boto3
from io import BytesIO
import pandas as pd
import numpy as np
from datetime import date
import zipfile
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings

warnings.filterwarnings("ignore")

print("Processo iniciado")

# Define diretorios
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

Processo iniciado


In [ ]:
# print("Carregando dados para previsões")

# # Prepara os dados para realizar as previsões
# zip_file = os.path.join(dataPath, 'estoque_da3.zip') # 14bi
# z = zipfile.ZipFile(zip_file)


# def ler_bases_exportadas(nome_arquivo):
#     z.extract(nome_arquivo)
#     df = pd.read_csv(nome_arquivo, sep=',')
#     os.remove(nome_arquivo)
#     return df


# base_conjunta = ler_bases_exportadas('estoque-divida.csv')

In [2]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

def read_s3(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

In [3]:
print("Carregando dados para previsões")

dados_predict = read_s3(
     bucket_name=os.getenv("S3_BUCKET_NAME"), 
     folder_name=os.getenv("S3_FOLDER_NAME"), 
     file_name='feature_store_igr_prime_ab1_maxpg.csv')

Carregando dados para previsões


In [4]:
dados_contribuinte = read_s3(
    bucket_name=os.getenv("S3_BUCKET_NAME"), 
    folder_name=os.getenv("S3_FOLDER_NAME"), 
    file_name='feature_store_contribuinte_prime_k4_maxpg.csv')

In [36]:
dados_contribuinte.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'valor_tot', 'valor_pago', 'qtd_notas_2anos', 'edificacao', 'situacao',
       'cpf_cnpj_existe', 'endereco_existe', 'perfil_acessivel',
       'situacao_cobranca', 'historico_pagamento_em_valor',
       'class_contribuinte', 'da_aberto', 'class_contribuinte_nome',
       'percentual_pago_cda', 'ordem_contrib', 'class_contribuinte_peso'],
      dtype='object')

In [10]:
formatar_moeda(dados_predict['valor_aberto'].sum())

'R$ 11.731.536.958,15'

In [ ]:
#print("Inicia transformação das variáveis sobre a dívida")
#print("Processamento das variáveis sobre a dívida")

# Gera as variáveis de tempo
#base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
#base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

In [13]:
# # Seleciona dados sobre a divida
# dados_divida = dados_predict[
#     ['cda', 'id_contribuinte', 'cpf_cnpj_existe',
#      'da_aberto', 'atividade_principal', 'valor_tot','vlr_pago', 'vlr_tributo', 'vlr_taxa', 'arrecadacao_divida',
#      'ajuizamento_divida', 'ajuizamento', 'protesto', 'competencia_divida', 'refis',
#      'tipo_divida', 
#      #'ano_inscricao_da', 
#      'quantidade_reparcelamento', 'situacao',
#      'anos_idade_da']]
# dados_divida.dropna(subset=['id_contribuinte'], inplace=True)
# dados_divida['id_contribuinte'] = dados_divida['id_contribuinte'].astype(str)  # persistindo tipo de dados

In [ ]:
# Calcula a idade da dívida ativa
#dados_divida['ano_atual'] = date.today().year
#dados_divida['anos_idade_da'] = dados_divida['ano_atual'] - dados_divida['ano_inscricao_da']
#dados_divida = dados_divida.drop(columns=['ano_atual'])

In [14]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
}
dados_predict = dados_predict.rename(columns=colunas_nome)

In [ ]:
# Obtem os dados de parcelamento das dívidas ativas

# df_parcelas = base_parcelas[['cda', 'id_pessoa', 'tipo_divida', 'quantidade_reparcelamentos']]
# df_parcelas.drop_duplicates(subset='cda', inplace=True)
# df_parcelas.dropna(subset='cda', inplace=True)

# df_divida_ativa = pd.merge(
#         df_divida_ativa,    
#         df_parcelas,    
#         how='left',    
#         left_on=['cda', 'tipo_divida', 'id_contribuinte'],    
#         right_on=['cda', 'tipo_divida', 'id_pessoa'],    
#         suffixes=('', '_PARC'))

In [15]:
# Filtrando variáveis de interesse para a modelagem
df_divida = dados_predict[['cda', 
                          'tipo_divida',
                          'id_pessoa', 
                          'da_aberto',
                          'percentual_pago_cda', 
                          'valor_total_da', 
                          'idade_divida'#, 
                          #'quantidade_reparcelamento' 
                          ]] 

In [37]:
df_contribuinte = dados_contribuinte[['id_pessoa', 
                                      'tipo_divida',
                                      'num_dist_cda', 
                                      'quantidade_reparcelamento',
                                      #'historico_pagamento_em_qtd',
                                       'historico_pagamento_em_valor',
                                       'situacao_cobranca',
                                       'class_contribuinte',
                                       'class_contribuinte_peso']]

In [38]:
df = pd.merge(left = df_divida, right = df_contribuinte, on=['id_pessoa', 'tipo_divida'])
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [ ]:
# print("Processamento das variáveis sobre o contribuinte")

# # Cria conexão com o banco e prepara os dados

# def read_s3_files(bucket_name, folder_name, file_name):
#     file_key_aws = folder_name + file_name
#     obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
#     df = pd.read_csv(obj['Body'], sep=';')
#     return df



# dados_contribuinte = read_s3(
#     bucket_name=os.getenv("S3_BUCKET_NAME"), 
#     folder_name=os.getenv("S3_FOLDER_NAME"), 
#     file_name='feature_store_contribuinte_prime_k4_maxpg.csv')


In [ ]:
# df_divida_ativa.columns

In [ ]:
# dados_contribuinte.columns

In [ ]:
# contribuinte_aux =  dados_contribuinte[['id_pessoa', 'edificacao',
#                                             'qtd_notas_2anos', 'situacao_ativa', 'status_situacao',
#                                              'frequencia_da_pessoa', 'historico_pagamento_em_qtd',
#                                              'historico_pagamento_em_valor', 'class_contribuinte',
#                                             'class_contribuinte_nome', 'class_contribuinte_peso']]

In [ ]:
# contribuinte_aux

In [ ]:
# # Realiza o merge dos dados da dívida com os dados do contribuinte

# df = pd.merge(
#     left=df_divida_ativa,
#     right=contribuinte_aux,
#     left_on='id_contribuinte',
#     right_on='id_pessoa',
#     how='left')

In [39]:
df.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'da_aberto', 'percentual_pago_cda',
       'valor_total_da', 'idade_divida', 'num_dist_cda',
       'quantidade_reparcelamento', 'historico_pagamento_em_valor',
       'situacao_cobranca', 'class_contribuinte', 'class_contribuinte_peso'],
      dtype='object')

In [22]:
# # Ajustando pesos dos contribuintes, foi dado ao primeira dívida o mesmo peso do bom pagador, isso procede?
# df.loc[df['frequencia_da_pessoa'] <= 1, 'class_contribuinte_nome'] == 'PRIMEIRA DIVIDA'
# df.loc[df['class_contribuinte_nome'] == 'PRIMEIRA DIVIDA', 'class_contribuinte_peso'] == 2.54487

# df['class_contribuinte_nome'] = df['class_contribuinte_nome'].fillna('PIOR PAGADOR')
# df.loc[df['class_contribuinte_nome'] == 'PIOR PAGADOR', 'class_contribuinte_peso'] == -0.98031
# df.fillna(0, inplace=True)

In [40]:
print("Preparação do pipeline de previsões")
# Filtra os dados que precisamos para previsão
df_feature_store = df[['valor_total_da', 
                       'idade_divida', #'anos_idade_da',
                        'quantidade_reparcelamento',
                       'num_dist_cda', #'frequencia_da_pessoa',
                        #'historico_pagamento_em_qtd', 
                        'situacao_cobranca', #'status_situacao',
                       'historico_pagamento_em_valor', 
                       'class_contribuinte_peso']]



Preparação do pipeline de previsões


In [41]:
# Normaliza os dados para previsão utilizando o StandardScaler
normalizador = StandardScaler() # RobustScaler()
normalizador.fit(df_feature_store)
dados_normalizados = normalizador.fit_transform(df_feature_store)

colunas = list(normalizador.get_feature_names_out())
df_normalizado = pd.DataFrame(dados_normalizados, columns=colunas)

In [42]:
print("Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)")


def abre_modelo(nome_modelo, path_modelo, zip_name=None):
    if zip_name:
        zip_file = os.path.join(path_modelo, zip_name)
        z = zipfile.ZipFile(zip_file)
        z.extract(nome_modelo)
    else:
        nome_modelo = os.path.join(path_modelo, nome_modelo)

    modelo = pickle.load(open(nome_modelo, 'rb'))
    return modelo


model_predict_igr = abre_modelo("modeloDA-igr-divida-prime_maxpg.pkl", modelsPath)

Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)


In [43]:
# Realizando previsões
previsoes = model_predict_igr.predict(df_normalizado)
df['igr'] = previsoes
df.loc[df['situacao_cobranca'] == 0, 'igr'] = 0
df.loc[df['idade_divida'] >= 15, 'igr'] = 0

print("Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte")

def make_rating_divida(dataframe):
    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Melhor Pagador
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'

    # Pior Pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Pagador intermediario
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Bom pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.05) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Primeira Dívida
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.3) & dataframe['igr'] >= 0.1, 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'
    return dataframe


df = make_rating_divida(df)

Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte


In [44]:
df

,cda,tipo_divida,id_pessoa,da_aberto,percentual_pago_cda,valor_total_da,idade_divida,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_valor,situacao_cobranca,class_contribuinte,class_contribuinte_peso,igr,rating_divida
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.000488,2048.73,1.0,3,0,0.00000,4.0,0,1.60383,0.369626,BAIXA
1,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.000488,2048.73,1.0,2,0,0.00000,4.0,0,1.60383,0.369626,BAIXA
2,2d144048238f263b6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.000866,1154.80,2.0,3,0,0.00000,4.0,0,1.60383,0.339500,BAIXA
3,2d144048238f263b6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,1,0.000866,1154.80,2.0,2,0,0.00000,4.0,0,1.60383,0.339500,BAIXA
4,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,1,0.000397,2515.85,1.0,5,0,0.00000,0.0,1,-1.21937,0.000000,BAIXISSIMA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517370,fffbc656a38a48fe9989bae6f4af91ee1,imovel,a408222571c32c15,1,0.010465,95.56,1.0,1,0,0.00000,4.0,4,-0.04597,0.815613,ALTA
1517371,fffc43ac68640f569fcacff532cf50c42,mercantil,7297863dc9809347,1,0.000045,22275.52,9.0,1,0,0.00000,0.0,4,-0.04597,0.000000,BAIXISSIMA
1517372,fffc43ac68640f569fcacff532cf50c42,mercantil,7297863dc9809347,1,0.000045,22275.52,9.0,1,0,0.00000,0.0,4,-0.04597,0.000000,BAIXISSIMA
1517373,fffe210df01bf5f52b46f673bf0c61251,imovel,485b19f05dea4e80,1,0.000271,3688.60,1.0,5,0,0.49278,4.0,0,1.60383,0.471832,BAIXA


In [50]:
metricas_predict = df.groupby('rating_divida')['cda'].nunique().to_frame().reset_index()
total = metricas_predict['cda'].sum()
metricas_predict['perc'] = np.round(metricas_predict['cda']/total, 5)
metricas_predict

,rating_divida,cda,perc
0,ALTA,76262,0.08469
1,ALTISSIMA,19079,0.02119
2,BAIXA,330598,0.36715
3,BAIXISSIMA,447720,0.49722
4,MEDIA,26787,0.02975


In [52]:
print("Inicia a conexão com S3 para inscrição dos dados com as previsões")

# Cria conexão ao s3 e preenche a tabela com os dados

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())


up_s3_files(dataframe=df,
            bucket_name=os.getenv("S3_BUCKET_NAME"),
            folder_name=os.getenv("S3_FOLDER_NAME"),
            file_name='prime_maxpg.csv')

print("Upload de dados efetuados no s3")
print("Processo finalizado")
print("Arquivo disponível para download e análise")

Inicia a conexão com S3 para inscrição dos dados com as previsões
Upload de dados efetuados no s3
Processo finalizado
Arquivo disponível para download e análise
